In [12]:
import git
import pandas as pd
g = git.Git("../../sonarqube/") 

In [13]:
csv_folder = "../csv"

In [14]:
tags_final = pd.read_csv(f"{csv_folder}/tags_comparison_final.csv", index_col=0)

In [15]:
data = []
for i, row in tags_final.iterrows():
    loginfo = g.log("--right-only", f'{row.from_sha}...{row.to_sha}')
    for line in loginfo.split("\n"):
        if line.startswith("commit"):
            data.append((row.from_tag, row.to_tag, line.replace("commit ", "")))

In [16]:
tags_commmits = pd.DataFrame(data, columns=["from_tag", "to_tag", "commitsha"])
tags_commmits.to_csv(f"{csv_folder}/tags_compare_commits.csv")

In [17]:
count = tags_commmits.groupby(["from_tag", "to_tag"]).count().reset_index()
count[count.from_tag.isin(tags_final[tags_final.behind_by>0].from_tag)]

,from_tag,to_tag,commitsha
10,5.3-RC1,5.3-RC2,40
29,5.4-RC2,5.4-RC3,11
30,5.4-RC2,5.5-M1,36
106,7.9,7.9.1,17
107,7.9,8.0,415
112,8.1.0.31237,8.2.0.32929,262


In [18]:
tags_final[tags_final.behind_by>0]

,from_tag,from_date,from_sha,to_sha,ahead_by,behind_by,to_tag,to_date,diff_dates,diff_days
2153,5.3-RC1,2015-12-09 12:47:34,c979578645aaf77c447b250136db32a7067619a1,29a84111f6cc9298f83df515830ff40a41d0976c,40,1,5.3-RC2,2015-12-17 16:02:01,8 days 03:14:27,8.135035
1669,5.4-RC2,2016-02-15 15:38:37,e8f951d9f2a99975e31f82d97fce68f47e9242d3,8909ccd99b9bc5874f24395519d66e2bcb9bdacb,36,1,5.5-M1,2016-02-12 15:43:47,-3 days +00:05:10,-2.996412
113,7.9,2019-07-01 07:56:26,6502ec4ffee7e5a57498baaf330b80eb490d01c2,87ca68d63f4afd37d74b2f454430dfde9e862c6a,415,1,8.0,2019-10-15 08:15:12,106 days 00:18:46,106.013032
41,8.1.0.31237,2019-12-17 12:29:34,65a7a7c34de5bff9aed7c3e58eb6dcc87ae34b57,749367809478e5b7f60c6ef952436db9789b5882,262,1,8.2.0.32929,2020-02-25 14:32:24,70 days 02:02:50,70.085301
